In [1]:
# Some base class that we need for this to work.  
class Z:
    pass

# This is like the model class, e.g. ReducedModel.  
class A(Z):
    def __new__(cls, backend): # We override the __new__ class with our own.  
                               # It does the hard work of setting the backend.  
        self = super().__new__(cls)
        print("Adding backend")
        bases = (backend.__class__,) + self.__class__.__bases__ # Add to the list of classes.  
                                                                # The set_backend in NeuronUnit basically does this.
        self.__class__.__bases__ = tuple(set(bases))
        return self
    def __init__(self, backend): # This basically just make sure the backend is stored as an 
                                 # attribute so the next method can find it.  
        self.backend = backend
        print("Initialized")
    def __getnewargs__(self): # This method is required by pickle to know what arguments to pass to __new__
                              # when instances of this class are eventually unpickled.  
                              # Otherwise __new__ will have no arguments.  
        return (self.backend,) # A tuple containing the extra arguments to pass to __new__.  
    
# This is the like the backend that we want to set.  
class B:
    def foo(self,x): # A method that we would like A to have when an instance of B is set as the backend.  
        print("x^2 = %d" % x**2)

my_backend = B() # Create an instance of the backend (this could be done inside A's __init__ as well).  
a = A(my_backend) # Create an instance of A with the backend that we want for it.  
a.foo(3) # Test to make sure the backend works.  

Adding backend
Initialized
x^2 = 9


In [2]:
import pickle
x = pickle.dumps(a) # Pickle.  
a2 = pickle.loads(x) # Unpickle. If we had not set the above cell correctly, this line would not work.  
a2.foo(4) # Test to make sure the backend still works.  

Adding backend
x^2 = 16


In [8]:
import os
import quantities as pq
import neuronunit
from neuronunit.models.reduced import ReducedModel
LEMS_MODEL_PATH = os.path.join(os.getcwd(),'docs/data/NeuroML2/LEMS_2007One.xml')
model = ReducedModel(LEMS_MODEL_PATH,name='vanilla',backend='jNeuroML')
current = {'amplitude':100*pq.pA}
model.inject_square_current(current)

In [9]:
import pickle
x = pickle.dumps(model)
model2 = pickle.loads(x)
model2.inject_square_current(current)